In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# 1. Cargar los datos
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

# 2. Preprocesamiento de texto: limpieza básica (minusculas y solo letras + espacios)
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    return text

train_data['message'] = train_data['message'].apply(clean_text)
test_data['message'] = test_data['message'].apply(clean_text)

# 3. Tokenización y padding
max_vocab = 10000
max_length = 100

tokenizer = Tokenizer(num_words=max_vocab, oov_token="<OOV>")
tokenizer.fit_on_texts(train_data['message'])

train_seq = tokenizer.texts_to_sequences(train_data['message'])
train_pad = pad_sequences(train_seq, maxlen=max_length, padding='post', truncating='post')

test_seq = tokenizer.texts_to_sequences(test_data['message'])
test_pad = pad_sequences(test_seq, maxlen=max_length, padding='post', truncating='post')

# 4. Mapear etiquetas a números
label_map = {'ham': 0, 'spam': 1}
train_labels = train_data['label'].map(label_map).values
test_labels = test_data['label'].map(label_map).values

# 5. Definir modelo mejorado con capas LSTM bidireccionales y dropout
model = Sequential([
    Embedding(max_vocab, 64, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(32)),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 6. Entrenamiento con EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit(train_pad, train_labels, epochs=5, batch_size=32,
          validation_data=(test_pad, test_labels), callbacks=[early_stop])

# 7. Función para predecir mensajes usando el modelo entrenado
def predict_message(pred_text):
    clean = clean_text(pred_text)
    seq = tokenizer.texts_to_sequences([clean])
    pad = pad_sequences(seq, maxlen=max_length, padding='post', truncating='post')
    prob_spam = float(model.predict(pad)[0][0])
    label = 'spam' if prob_spam > 0.5 else 'ham'
    prob_ham = 1 - prob_spam
    return [prob_ham, label]



# Ejemplo de prueba
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
